# Setup tests

In [1]:
import ipytest
ipytest.autoconfig()

In [2]:
%%ipytest -vvv -s
from test_rdf_files import *
from test_rdf_parsing import *
from test_include_cycle import *
from test_consistency import *
from test_rdf_samples import *


======================================= test session starts =======================================
platform win32 -- Python 3.13.1, pytest-8.4.2, pluggy-1.6.0 -- c:\Programs\Python313\python.exe
cachedir: .pytest_cache
rootdir: e:\users\tel\Python\site-packages\Scriptum
configfile: pyproject.toml
collecting ... collected 17 items

t_f9268f53c1ce491aa293ed04ba81ec15.py::test_marker_behavior <- tests\02_basetest\rdf\test_rdf_files.py PATH C:\Users\tel\AppData\Local\Temp\pytest-of-TEL\pytest-48\test_marker_behavior0 foo
PASSED
t_f9268f53c1ce491aa293ed04ba81ec15.py::test_multi_section_paths_and_consistency <- tests\02_basetest\rdf\test_rdf_files.py PASSED
t_f9268f53c1ce491aa293ed04ba81ec15.py::test_repeat_section_generates_unique_addresses <- tests\02_basetest\rdf\test_rdf_files.py PASSED
t_f9268f53c1ce491aa293ed04ba81ec15.py::test_include_combines_tasks <- tests\02_basetest\rdf\test_rdf_parsing.py PASSED
t_f9268f53c1ce491aa293ed04ba81ec15.py::test_recursion_depth_limit <- tests\02_basete

In [3]:
stop here

SyntaxError: invalid syntax (4067800170.py, line 1)

In [ ]:
mylist = list(range(10))
print(mylist)
five = next(v for v in mylist if v == 5)
print(five, mylist)
six = next(v for v in mylist if v == 6)
print(six)

In [ ]:
def writeTestData(filename,data):
    with open(filename,'w') as f:
        f.write(data)
def runTest(filename, testmode=False,debug=False):
    importlib.reload(reportDataFile)
    reportDataFile.ReportDataFile._testmode_=testmode
    rdf=reportDataFile.ReportDataFile(filename, debug=debug)
    return rdf

In [ ]:
# check for replications and serials
data = """*version=100
*documenttype=docx
# not allowed
# .test='Foo'
#section:a.title='Bar'
section:a
 .subsection:load
 .subsubsection:test
  .head='Concentrated Force'
 .subsection:load
    .head='Temperatures'
# # replicate
section:a
   .subsection:load
#    .head='Concentrated Force'
#section:b
#  .head='S b'
#   .subsection:load
#  .subsubsection:test
#   .head='Concentrated Force'
#  section:a.subsection:load.subsubsection:test
#   .head='Concentrated Force'
#  .subsubsection:test
#   section:a.subsection:load.subsubsection:test.head='Concentrated Force'
  """

writeTestData('repeatSection.rdf', data)
rdf = runTest('repeatSection.rdf', True,False)

#for task in rdf.tasks:
#    print(task)
rdf.inspect()

In [ ]:
tg = rdf.groupMe()

In [ ]:
print(tg)

In [ ]:
tg.groupOf[2].groupOf[1]

In [ ]:
rdf.tasks[0]._allPaths

In [ ]:
rdf.tasks[0]._newPaths

In [ ]:
# check for replications and serials
data = """*version=100
*documenttype=pptx
slide:a
slide:a.slide:load
  .head='Concentrated Force'
   .float=1.4
  .integer=42
  .head='Temperatures'
  .type='Encastre'
# replicate
a
slide:a.load.other
  .head='Concentrated Force'
  .float=1.4
  .integer=42
  .head='Temperatures'
  .type='Encastre'
"""

writeTestData('repeatSection.rdf', data)
rdf = runTest('repeatSection.rdf', True,True)

for task in rdf.tasks:
    print(task)
#rdf.inspect()

In [ ]:
# check for sections, even recursive
data = """*version=100
*documenttype=docx
global
.setup=1
section:load_bc
.subsection:load
.head='Concentrated Force'
.float=1.4
.integer=42
.text:description='This load adds a single load at the front'"""
writeTestData('multiSection.rdf', data)
rdf = runTest('multiSection.rdf', True,True)
print(rdf)
rdf.inspect()

In [ ]:
str(rdf.tasks[1].value)

In [ ]:
# check @, +, and &
data="""*version=100
*documenttype=docx
*datadir=data
# there is one subsection:load already in the document
section:load_bc.subsection:load
  *datadir=foo
  .parameter=parfile:foo.nv:'Hello world parameter'
  .head='Pressure Load 1'
  #&include=loopfiles:analysis*.rdf
  @marker:content
    +image:generic=file:load1.png+description='load one'
    +image:generic=file:load2.png+description='load two'
    +image:generic=file:load3.png+description='load three'
    #&include=loopfiles:bou*.rdf
"""
writeTestData('testMarker.rdf', data)
rdf = runTest('testMarker.rdf', True, True)
#print(rdf)
rdf.inspect()
#rdf.showFiles()

In [ ]:
# check wether last result will replicate
data=repr(rdf)
writeTestData('resource.rdf',data)
rdfre = runTest('resource.rdf',True)
print(rdfre)

In [ ]:
# check date and time
# 
data="""*version=100
*dateformat=%x
*timeformat = %X
*datetimeformat = %c %c
mysection.subsection
  .setdate=date:today
  .settime=date:now
  .testtime=date:12/15/22 14:24:6933 abc
  .created=date:now:'%d. %b %Y -- %H:%M:%S'
  .initial=date:1231231230:'%d. %b %Y -- %H:%M:%S'
  .toolong=date:12312312345689:'%d. %b %Y -- %H:%M:%S'
  .tooshort=date:123123:'%d. %b %Y -- %H:%M:%S'
"""
writeTestData('testTimeDate.rdf', data)
rdf = runTest('testTimeDate.rdf', True)
rdf.inspect()
#str(rdf.tasks[4].value)

In [ ]:
# check numbering
# 
data="""*version=100
mysection.subsection
  .number=numbering:1:%s):2
  .roman=numbering:I:%s):;:A;B;C;D;E;K
  .lowroman=numbering:i:%s ):3
  .char=numbering:A:%s ):3
  .lowchar=numbering:a:%s ):6
"""
writeTestData('testNumbers.rdf', data)
rdf = runTest('testNumbers.rdf', True)
rdf.inspect()
#print(next(rdf.tasks[0].value.value))
#print(next(rdf.tasks[0].value.value))
#print(next(rdf.tasks[2].value.value))
#print(next(rdf.tasks[3].value.value))
#print(next(rdf.tasks[4].value.value))

In [ ]:
for t in rdf.tasks:
    print(t)

In [ ]:
# check length
# 
data="""*version=100
mysection.subsection
  .inch=1in
  .centimeter=12.3cm
  .point=15pt
  .millimeter=123.56mm
  .wrong=123.3.4mm
"""
writeTestData('testLength.rdf', data)
rdf = runTest('testLength.rdf', True)
rdf.inspect()

In [ ]:
# check for recursivion limit
data = """*version=100
section:load_bc
&include+=file:multiSection.rdf"""
writeTestData('multiSection.rdf', data)
rdf = runTest('multiSection.rdf', True)
print(rdf)
rdf.inspect()

In [ ]:
rawtask = "hallo.welt+table:generic=file:table3.csv+description=@row1"
elements = rawtask.split('+')
print(elements)
path = elements[0].strip()
print('p',path)
addition = elements[1].split('=')
print('a',addition)
basepath = path+'.'+addition[0].strip()
print('bp',basepath)
actions = {}
for el in elements[2:]:
    n,v = el.split('=')
    n = n.strip()
    actions[n] = v
print(actions)
print(basepath+'='+addition[1])
#task = [ReportTask(basepath+'='+addition[1], self.settings, **modifier)]    
